# Props Data Organizer

## Definitions of Formulas Used in This Notebook

### Estimated Value (EV)
EV is the average amount you can expect to win or lose per bet if you placed the same bet many times. It helps identify profitable betting opportunities by comparing the expected return to the risk involved.

**Formula:**
$$
\text{EV} = (\text{Probability of Winning} \times \text{Profit if Win}) - (\text{Probability of Losing} \times \text{Loss if Lose})
$$

### Kelly Criterion
The Kelly Criterion is a formula used to determine the optimal size of a series of bets. It aims to maximize the logarithm of wealth, balancing the trade-off between risk and reward. The formula considers both the probability of winning and the odds offered, guiding you on how much of your bankroll to wager on each bet.

**Formula:**
$$
\text{Kelly Fraction} = \frac{(\text{Probability of Winning} \times (\text{Odds} + 1)) - 1}{\text{Odds}}
$$

In [2]:
import pandas as pd 
import numpy as np
import time
import requests
from NBAData.gambling import *
from datetime import datetime
from Models.xgboost_prediction import *

today = datetime.now()
formatted_date = today.strftime("%m_%d_%y")

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Grabs players odds for the day (US all boookmakers, DFS is prizepicks and underdogs)

In [ ]:
from NBAPropFinder.NBAPropFinder import NBAPropFinder

nba_props = NBAPropFinder(region='us_dfs')
prizePicks = nba_props.dataframe
prizePicks.head(10)

In [2]:
from WNBAPropFinder.WNBAPropFinder import WNBAPropFinder
wnba_props = WNBAPropFinder(region='us')
prizePicks = wnba_props.dataframe
prizePicks

Scraping Odds API...
Organizing Data...


,BOOKMAKER,CATEGORY,NAME,OVER/UNDER,LINE,ODDS
0,FanDuel,player_points,Tina Charles,Over,16.5,-128
1,FanDuel,player_points,Tina Charles,Under,16.5,-102
2,FanDuel,player_points,Chelsea Gray,Over,12.5,-114
3,FanDuel,player_points,Chelsea Gray,Under,12.5,-114
4,FanDuel,player_points,Jewell Loyd,Over,11.5,-106
...,...,...,...,...,...,...
427,FanDuel,player_rebounds_assists,Caitlin Clark,Under,15.5,-136
428,FanDuel,player_rebounds_assists,Napheesa Collier,Over,13.5,106
429,FanDuel,player_rebounds_assists,Napheesa Collier,Under,13.5,-140
430,FanDuel,player_rebounds_assists,Caitlin Clark,Over,14.5,-130


### Single Bets from bookmakers that dont include prizePicks or UnderDogs

In [10]:
model = loadXGBModel('PTS')
bookmakers = pd.read_csv(f'CSV_FILES/PROPS_DATA/Playoffs_US(06_22_25).csv')
data = pd.read_csv('CSV_FILES/PLAYOFF_DATA/PLAYOFFS_25_PTS_FEATURES.csv')
games = get_espn_games(date_str='20250622')



In [9]:
# Dictionary mapping prop categories to their stat columns
propDict = {
    'player_points': 'PTS',
    'player_rebounds': 'REB',
    'player_assists': 'AST',
    # 'player_threes': 'FG3M',
    # 'player_blocks': 'BLK',
    # 'player_steals': 'STL',
    # 'player_field_goals': 'FGM',
    # 'player_threes': 'FG3M',
    # 'player_frees_made': 'FTM',
    # 'player_frees_attempts': 'FTA',
    # 'player_turnovers': 'TOV',
    # 'player_points_rebounds_assists': 'PTS+REB+AST',
    # 'player_points_rebounds': 'PTS+REB',
    # 'player_points_assists': 'PTS+AST',
    # 'player_rebounds_assists': 'REB+AST',
    # 'player_blocks_steals': 'BLK+STL'
}
models = {
    'PTS': loadXGBModel('PTS'),
    'REB': loadXGBModel('REB'),
    'AST': loadXGBModel('AST'),
}
all_results = []

for category, stat in propDict.items():
    print(f"Processing {category}...")
    data = pd.read_csv(f'CSV_FILES/PLAYOFF_DATA/PLAYOFFS_25_{stat}_FEATURES.csv')
    results = single_bet(data, bookmakers, models, games, category=category, stat_line=stat)
    all_results.append(results)

combined_results = pd.concat(all_results, ignore_index=True)

final_results = combined_results.sort_values(by='EV', ascending=False).reset_index(drop=True)

print("\nTop 15 highest EV bets across all prop types:")
final_results.head(15)

Processing player_points...
Processing single bets...
Loaded dataset for PTS
Processing player_rebounds...
Processing single bets...
Loaded dataset for REB
Processing player_assists...
Processing single bets...
Loaded dataset for AST

Top 15 highest EV bets across all prop types:


,NAME,TEAM,OPPONENT,BOOKMAKER,CATEGORY,LINE,OVER/UNDER,ODDS,FAIR ODDS,MODEL_PREDICTION,SIM_PROB,EV,KELLY CRITERION
0,Tyrese Haliburton,IND,OKC,Bovada,player_assists,5.5,Under,205,239,13,0.996,203.69,0.9957
1,Myles Turner,IND,OKC,Bovada,player_points,9.5,Under,215,251,15,0.900,183.59,0.8998
2,T.J. McConnell,IND,OKC,Bovada,player_assists,3.5,Under,165,190,7,0.972,157.55,0.9717
3,Aaron Nesmith,IND,OKC,Bovada,player_points,7.5,Under,215,251,11,0.812,155.94,0.8116
4,Tyrese Haliburton,IND,OKC,Bovada,player_points,12.5,Under,190,228,19,0.867,151.55,0.8667
5,Bennedict Mathurin,IND,OKC,Bovada,player_points,5.5,Under,165,190,13,0.917,143.11,0.9169
6,T.J. McConnell,IND,OKC,Bovada,player_assists,5.5,Over,200,239,7,0.787,136.04,0.7857
7,Tyrese Haliburton,IND,OKC,Bovada,player_assists,7.5,Over,125,139,13,0.972,118.81,0.9723
8,Myles Turner,IND,OKC,Bovada,player_points,10.5,Under,155,182,15,0.856,118.18,0.8547
9,Tyrese Haliburton,IND,OKC,FanDuel,player_assists,7.5,Over,124,139,13,0.973,117.88,0.9725


In [ ]:
from Models.xgboost_prediction import *
from Models.xgboost_model import *
from NBAData.gambling import *


player = "Anthony Davis"
modelPTS = loadXGBModel(stat_line='PTS')
modelAST = loadXGBModel(stat_line='AST')
modelREB = loadXGBModel(stat_line='REB')

bookmakers = pd.read_csv('CSV_FILES/HISTORICAL_ODDS/ALL_HISTORICAL_ODD.csv')
PrizePicks = bookmakers[(bookmakers['BOOKMAKER'] == 'PrizePicks') & 
                        (bookmakers['commence_time'] == '2024-10-22')]

backTestData = pd.read_csv('CSV_FILES/REGULAR_DATA/SEASON_25_PTS_FEATURES.csv')
player_df = backTestData[backTestData['PLAYER_NAME'] == player].sort_values('GAME_DATE')
backTestData = backTestData[backTestData['GAME_DATE'] == '2024-10-22']

games = get_espn_games(date_str='20241022')
opponent = findOPP(player, backTestData, games)
endOfSeasonPTS = pd.read_csv('CSV_FILES/REGULAR_DATA/SEASON_24_PTS_FEATURES.csv')
endOfSeasonAST = pd.read_csv('CSV_FILES/REGULAR_DATA/SEASON_24_AST_FEATURES.csv')
endOfSeasonREB = pd.read_csv('CSV_FILES/REGULAR_DATA/SEASON_24_REB_FEATURES.csv')

# predPTS = make_prediction(
#     player_name=player,
#     bookmakers=PrizePicks,
#     opponent=opponent, 
#     model=modelPTS, 
#     data=endOfSeasonPTS, 
#     games=games, 
#     is_playoff=0,
#     stat_line='PTS')
# monte_carlo_prop_simulation(
#     player_df=player_df,
#     modelPred=predPTS['predicted_stat'],
#     prop_line=predPTS['prop_line'],
#     stat_line='PTS'
# )

In [ ]:
propDict = {
    'player_points': 'PTS',
    'player_rebounds': 'REB',
    'player_assists': 'AST',
}
models = {
    'PTS': loadXGBModel('PTS'),
    'REB': loadXGBModel('REB'),
    'AST': loadXGBModel('AST'),
}
pairs = prizePicksPairsEV(PrizePicks, propDict, models, games, simulations=10000, stake=100, payout=300)
pairs.sort_values('EV', ascending=False).head(5).reset_index(drop=True)

In [ ]:
trios = prizePicksTriosEV(PrizePicks, propDict, models, games, simulations=10000, stake=100, payout=300)
trios.sort_values('EV', ascending=False).head(5).reset_index(drop=True)